In [1]:
from pathlib import Path

# Load the original CSS file
css_path = Path("../web/static/css/style.css")
css_content = css_path.read_text()

# Define new color scheme for blood donation website
color_map = {
    "#060dcfc7": "#bb0a1e",     # Replace deep blueish header backgrounds with Blood Red
    "#e3e7ee": "#fff5f5",       # Replace light gray backgrounds with soft white/pinkish
    "#ffffff": "#fff",         # Keep pure white
    "#9a82842e": "#8b000022",  # Subtle shadowing replaced with dark red alpha
    "#dbdade": "#dc143c33",    # Box shadow gray to crimson alpha
    "#060dcf": "#bb0a1e",      # Any other reference to the blue to blood red
    "#060DCF": "#bb0a1e",
    "#060dcf": "#bb0a1e",
    "#f2f2f2": "#800000",      # Border to maroon
    "#ee6e73": "#dc143c",      # Red for quote border left
    "#9acd32": "#bb0a1e",      # LimeGreen replaced with Blood Red
    "#f8ce0b": "#bb0a1e",      # Yellow stars replaced with Blood Red
    "#b5b5b5": "#800000",      # Neutral gray background to maroon
    "#9c9393ba": "#800000aa",  # Shadow to maroon tone
    "#9a82847a": "#8b000077",  # Shadow color to dark red
}

# Apply the color replacements
for old_color, new_color in color_map.items():
    css_content = css_content.replace(old_color, new_color)

# Save the updated CSS content
updated_css_path = "../web/static/css/new_style.css"
Path(updated_css_path).write_text(css_content)

updated_css_path


'../web/static/css/new_style.css'

In [2]:
from pyflowchart import *

start_prep_faq = StartNode("Prepare FAQ DB")
load_base_faq_db = InputOutputNode(InputOutputNode.OUTPUT,"Save base question/answer pairs to the DB")
generate_questions = OperationNode("Generate additional questions")
concatenate_q_a = OperationNode("Concatenate generated questions + answers")
embed_qa = OperationNode("Embed Q&A using text-embedding-3-large model")
save_embeddings = InputOutputNode(InputOutputNode.OUTPUT,"Save embeddings as Numpy blob in the DB")
end_prep_faq = EndNode("Prepare FAQ DB")
# connect graph
start_prep_faq.connect(load_base_faq_db)
load_base_faq_db.connect(generate_questions)
generate_questions.connect(concatenate_q_a)
concatenate_q_a.connect(embed_qa)
embed_qa.connect(save_embeddings)
save_embeddings.connect(end_prep_faq)
prep_faq_chart = Flowchart(start_prep_faq)
output_html('prep_faq_chart.html', 'Prepare FAQ DB', prep_faq_chart.flowchart())


Saved HTML to prep_faq_chart.html


In [ ]:
# Search flow graph
from pyflowchart import *

message = InputOutputNode(InputOutputNode.INPUT, "user: E.g. How often can I donate blood?")
start_search_flow = StartNode("search flow")
embed_message = OperationNode("Embed user message")
search_faiss = OperationNode("Search in FAISS in-memory vector store")
search_results = InputOutputNode(InputOutputNode.OUTPUT, "Pointers to 3 most relevant FAQ responses")
evaluate_search_results = OperationNode("Select the most relevant response or generate new one")
return_search_response = InputOutputNode(InputOutputNode.OUTPUT, "Selected or generated response")
#action_search_faiss = NodesGroup(embed_message,[search_faiss,search_results, evaluate_search_results, return_search_response])
ask_for_feedback = ConditionNode("Was the answer helpful? Yes or No")
sorry_could_not_help = InputOutputNode(InputOutputNode.OUTPUT, "bot: Thank you for feedback, I am sorry I could not help you")
thank_for_feedback = InputOutputNode(InputOutputNode.OUTPUT,"bot: I am glad for helping you")
save_feedback = OperationNode("Save feedback to DB")
how_can_i_help = InputOutputNode(InputOutputNode.OUTPUT,"bot: How can I further help you?")
end_search_flow = EndNode("search flow")
# connect graph
message.connect(start_search_flow)
start_search_flow.connect(embed_message)
embed_message.connect(search_faiss)
search_faiss.connect(search_results)
search_results.connect(evaluate_search_results)
evaluate_search_results.connect(return_search_response)
return_search_response.connect(ask_for_feedback)
ask_for_feedback.connect_yes(thank_for_feedback, direction="right")
thank_for_feedback.connect(save_feedback)
ask_for_feedback.connect_no(sorry_could_not_help,direction="left")
sorry_could_not_help.connect(save_feedback)
save_feedback.connect(how_can_i_help)
how_can_i_help.connect(end_search_flow)

search_flow_chart = Flowchart(message)
output_html('search_flow.html', 'Search FAQ flow', search_flow_chart.flowchart())


Saved HTML to search_flow.html


In [18]:
# Book donation flow chart
message = InputOutputNode(InputOutputNode.INPUT,"user: I want to book a blood donation")
start_booking_flow = StartNode("Book donation flow")
ask_auth_continue = InputOutputNode(InputOutputNode.OUTPUT,"bot: I have to authenticate you before we can proceed. Will you continue?")
condition_auth_continue = ConditionNode("Continue:")
sub_authenticate_flow = SubroutineNode("flow_authenticate_user")
condition_authenticated = ConditionNode("Authenticated")
how_can_i_help_auth = InputOutputNode(InputOutputNode.OUTPUT,"bot: How can I further help you?")
access_code_invalid = InputOutputNode(InputOutputNode.OUTPUT, "bot: The access code is not valid.")
ask_select_donation_centre = InputOutputNode(InputOutputNode.OUTPUT, "bot: Please select your preferred donation centre")
donation_centre_selected = InputOutputNode(InputOutputNode.INPUT, "user: <donation centre>")
ask_select_date = InputOutputNode(InputOutputNode.OUTPUT,"bot: Please select your preferred date")
selected_date = InputOutputNode(InputOutputNode.INPUT, "user: <selected date>")
ask_select_time_slot = InputOutputNode(InputOutputNode.OUTPUT, "Please select your preferred time slot")
selected_time_slot = InputOutputNode(InputOutputNode.INPUT, "user: <time slot>")
booking_confirmed = InputOutputNode(InputOutputNode.OUTPUT,"bot: Your booking has been confirmed")
send_booking_confirmation = OperationNode("Booking confirmation mail with ics sent to user")
ask_fill_questionnaire = InputOutputNode(InputOutputNode.OUTPUT,"Do you want to fill out the pre-donation questionnaire?")
condition_fill_questionnaire = ConditionNode("Fill questionnaire:")
fill_questionnaire_yes = SubroutineNode("Fill questionaire flow")
booking_flow_end = EndNode("Book donation flow")
how_can_i_help_auth = InputOutputNode(InputOutputNode.OUTPUT,"bot: How can I further help you?")
how_can_i_help_access_code = InputOutputNode(InputOutputNode.OUTPUT,"bot: How can I further help you?")
how_can_i_help_questionnaire = InputOutputNode(InputOutputNode.OUTPUT,"bot: How can I further help you?")

message.connect(start_booking_flow)
start_booking_flow.connect(ask_auth_continue)
ask_auth_continue.connect(condition_auth_continue)
condition_auth_continue.connect_yes(sub_authenticate_flow)
sub_authenticate_flow.connect(condition_authenticated)
condition_authenticated.connect_yes(ask_select_donation_centre)
condition_authenticated.connect_no(how_can_i_help)
condition_auth_continue.connect_no(how_can_i_help_auth)
ask_select_donation_centre.connect(donation_centre_selected)
donation_centre_selected.connect(ask_select_date)
ask_select_date.connect(selected_date)
selected_date.connect(ask_select_time_slot)
ask_select_time_slot.connect(selected_time_slot)
selected_time_slot.connect(booking_confirmed)
booking_confirmed.connect(send_booking_confirmation)
send_booking_confirmation.connect(ask_fill_questionnaire)
ask_fill_questionnaire.connect(condition_fill_questionnaire)
condition_fill_questionnaire.connect_yes(fill_questionnaire_yes)
condition_fill_questionnaire.connect_no(how_can_i_help_questionnaire)
how_can_i_help.connect(booking_flow_end)
how_can_i_help_auth.connect(booking_flow_end)
how_can_i_help_access_code.connect(booking_flow_end)
how_can_i_help_questionnaire.connect(booking_flow_end)

fill_questionnaire_yes.connect(booking_flow_end)

book_donation_flow_chart = Flowchart(message)
output_html('book_donation_flow.html', 'Book donation  flow', book_donation_flow_chart.flowchart())

Saved HTML to book_donation_flow.html


In [15]:
## Authenticate user flow chart
# Nodes
start_flow_authenticate_user = StartNode("flow_authenticate_user")
ask_input_email = InputOutputNode(InputOutputNode.OUTPUT,"bot: Please type in your e-mail")
input_email = InputOutputNode(InputOutputNode.INPUT,"user: <email>")
sub_authenticate_flow = SubroutineNode("flow_authenticate_user")
validate_email = OperationNode("Check e-mail in DB")
condition_valid_email = ConditionNode("E-mail valid")
email_invalid = InputOutputNode(InputOutputNode.OUTPUT,"bot: Your e-mail was not found in the DB, do you want to register?")
sub_register_flow = SubroutineNode("flow_register_user")
ask_input_access_code = InputOutputNode(InputOutputNode.OUTPUT,"bot: I've sent an access code to your e-mail address. Please type in the access code.")
input_access_code = InputOutputNode(InputOutputNode.INPUT, "user: <access code>")
validate_access_code = OperationNode("Validate access code in the DB")
condition_access_code = ConditionNode("Access code valid")
end_auth_user_flow = EndNode("flow_authenticate_user")
# Edges
start_flow_authenticate_user.connect(ask_input_email)
ask_input_email.connect(input_email)
input_email.connect(validate_email)
validate_email.connect(condition_valid_email)
access_code_invalid.connect(how_can_i_help_access_code)
condition_valid_email.connect_yes(ask_input_access_code)
condition_valid_email.connect_no(email_invalid)
email_invalid.connect(sub_register_flow)
sub_register_flow.connect(end_auth_user_flow)
ask_input_access_code.connect(input_access_code)
input_access_code.connect(validate_access_code)
validate_access_code.connect(condition_access_code)
condition_access_code.connect_no(access_code_invalid)
condition_access_code.connect_yes(end_auth_user_flow)
access_code_invalid.connect(end_auth_user_flow)

flow_authenticate_user_chart = Flowchart(start_flow_authenticate_user)
output_html('flow_authenticate_user.html', 'Authenticate user   flow', flow_authenticate_user_chart.flowchart())

Saved HTML to flow_authenticate_user.html
